# Solving text similarity problem

The text similarity problem deals with the challenge of finding how close given text documents are.

In this part we will take a look into how to solve similarity problem using TF-IDF (Term Frequency Inverse Document Frequency)

Let's understand this by depicting into TF and IDF:
- TF (Term Frequency): is a tehnique to find the frequency of a word in a given document
Here, we normalize the frequency with respect to the total words that are present in the document and compute the TF value of a word

- IDF (Inverse document frequency): is a tehnique in which we make sure that words that are frequently used (a, the, and so on) are given a lower weight when compared to words that are rarely used.

In [1]:
# import required libraries
import nltk
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
class TextSimilarity:
    
    """
    (1) - define constructor which takes sample sentences on which we want to find the similarity
    """
    
    def __init__(self):
        self.statements = [
            "The sky is blue and beautiful.",
            "Love this blue and beautiful sky!",
            "The quick brown fox jumps over the lazy dog.",
            "A king’s breakfast has sausages, ham, bacon, eggs, toast and beans",
            "I love green eggs, ham, sausages and bacon!",
            "The brown fox is quick and the blue dog is lazy!",
            "The sky is very blue and the sky is very beautiful today",
            "The dog is lazy but the brown fox is quick!",
            "President greets the press in Chicago",
            "Obama speaks in Illinois"
        ]
        
    """ 
    (2) - define TF() method which takes as input a sentence.
        - convert the sentce to lower case and extract all words
        - find frequency distribution of these words using nltk FreqDist function
        - iterate over all dictionary keys, build the normalized floating values and store them into a dictionary
        - return dictionary that contains the normalized score for each word in the sentence
    """
    
    def TF(self, sentence):
        """
        Method to return a dictionary where key is the word and value its frequency.
        """
        
        # tokenize sentence and lower case each word
        words = nltk.word_tokenize(sentence.lower())

        # get frequency distribution
        freq = nltk.FreqDist(words)
        
        # define an empty dictionary
        dictionary = {}
        
        # iterate over all frequency distribution keys
        for key in freq.keys():
            # build TF for each word by normalizing floating values
            norm = freq[key]/float(len(words))
            dictionary[key] = norm
        
        return dictionary

    """
    (3) - define IDF() method that finds the IDF value for all the words in all documents.
        - define a local function caled idf() representing the formula for finding IDF of a given word
        - iterate over all the statements and convert them to lowercase
        - find the number of occurences of each word that is present across all the documents
        - build the IDF value for all words and return the dictionary containing these IDF values
    """
    def IDF(self):
        """
        Method to find and return the IDF for all the words in all documents.
        """
        def idf(TotalNumberOfDocuments, NumberOfDocumentsWithThisWord):
            """
            Local function to find the IDF of a given word.
            """
            return 1.0 + math.log(TotalNumberOfDocuments/NumberOfDocumentsWithThisWord)
        
        numDocuments = len(self.statements)
        uniqueWords = {}
        idfValues = {}
        
        for sentence in self.statements:
            for word in nltk.word_tokenize(sentence.lower()):
                # find how many times each word is present across all documents
                if word not in uniqueWords:
                    uniqueWords[word] = 1
                else:
                    uniqueWords[word] += 1
        
        # build IDF value for all words
        for word in uniqueWords:
            idfValues[word] = idf(numDocuments, uniqueWords[word])
        return idfValues
    
    """
    (4) - define a method to perform TF*IDF for all documents against a given search string.
        - break the string into tokens
        - build IDF() for all sentences in the self.statements variable
        - iterate over all sentences and find the TF for all words in the sentence
        - filter and use only the words that are present in the input search string 
        - build the vectors that consist of ff*idf values against each document
        - return the list of vectors for each word in the search query
    """
    def TF_IDF(self, query):
        """
        Method to perform TF * IDF for all documents against a given search string 
        and return the list of vectors for each word in the search query.
        """
        
        # break the search string into tokens
        words = nltk.word_tokenize(query.lower())
        
        # build IDF() for all sentences
        idf = self.IDF()
        
        vectors = {}
        # iterate over all sentences 
        for sentence in self.statements:
            # find all TF for all words in the sentence
            tf = self.TF(sentence)
            # filter and use only the words that are present in the input search string
            for word in words:
                tf_vector = tf[word] if word in tf else 0.0
                idf_vector = idf[word] if word in idf else 0.0
                # build tf*idf
                multiplication = tf_vector * idf_vector
                if word not in vectors:
                    vectors[word] = []
                else:
                    vectors[word].append(multiplication)
        
        return vectors
    
    """
    (5) - define function to display the contents of vectors on the screen
    """
    def displayVectors(self, vectors):
        print(self.statements)
        for word in vectors:
            print("{} -> {}".format(word, vectors[word]))
    
    
    """
    (6) - define function to compute cosine similarity score
        - create a new vectorizer object
        - build matrix of TF-IDF values for all the documents that we are interested in using fit_transform() function
    """
    def cosineSimilarity(self):
        vec = TfidfVectorizer()
        matrix = vec.fit_transform(self.statements)
        for j in range(1, 5):
            i = j - 1
            print("\tsimilarity of document {} with others".format(i))
            similarity = cosine_similarity(matrix[i:j], matrix)
            print(similarity)
            
    """
    (7) - define run function 
        - take the first statement as input query to compare with other sentences
        - display TF*IDF vectors for all sentences on screen
        - print consine similarity computed for all the sentences usign scikit library
    """      
    def run(self):
        inputQuery = self.statements[0]
        vectors = self.TF_IDF(inputQuery)
        self.displayVectors(vectors)
        self.cosineSimilarity()
      

In [15]:
similarity = TextSimilarity()
similarity.run()

['The sky is blue and beautiful.', 'Love this blue and beautiful sky!', 'The quick brown fox jumps over the lazy dog.', 'A king’s breakfast has sausages, ham, bacon, eggs, toast and beans', 'I love green eggs, ham, sausages and bacon!', 'The brown fox is quick and the blue dog is lazy!', 'The sky is very blue and the sky is very beautiful today', 'The dog is lazy but the brown fox is quick!', 'President greets the press in Chicago', 'Obama speaks in Illinois']
the -> [0.0, 0.2, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.18181818181818182, 0.16666666666666666, 0.0]
sky -> [0.273755818839165, 0.0, 0.0, 0.0, 0.0, 0.3193817886456925, 0.0, 0.0, 0.0]
is -> [0.0, 0.0, 0.0, 0.0, 0.2261124906564554, 0.2261124906564554, 0.2466681716252241, 0.0, 0.0]
blue -> [0.273755818839165, 0.0, 0.0, 0.0, 0.15969089432284625, 0.15969089432284625, 0.0, 0.0, 0.0]
and -> [0.2158322319665701, 0.0, 0.08887209551564651, 0.13734778397872643, 0.12590213531383254, 0.12590213531383254, 0.0, 0.0, 0.0]
beautif